# Profit Optimization
Another typical logistic problem for a grocer is to maximize overall profit by selecting an optimal set of inventory. To build the model for this problem, assumptions should be given at the very beginning:

  1. there are $N$ different kinds of items $U=\{ U_0, U_1,\cdots,U_{N-1} \}$
  2. each of the item has its own selling price $V=\{ V_0, V_1, \cdots, V_{N-1}\}$
  3. each of the item has its own cost $W=\{ W_0, W_1, \cdots, W_{N-1}\}$
  4. quantity of buying each item has an upper bound $B=\{ B_0, B_1, \cdots, B_{N-1}\}$ 
  5. there is a total budget limit $C$ for restoring inventory
  6. all items will be sold out

This is a "Knapsack" problem, and D-Wave CQM solver is uised to handle this problem.

## Import Labraries

In [1]:
from dimod import CQM, Binary, Integer, quicksum
from dwave.system import LeapHybridCQMSampler
import numpy as np

## Problem Set Up
Randomly generating list of item values $V$, item costs $W$, quantity upper bounds $B$, and the budget limit $C$.

In [2]:
# ---------- Problem set up -------------
num_of_items = 12
# values of every item (e.g. [8, 7, 1, 8, 5, 9, 8, 7, 1, 4, 1, 3])
V = list(np.random.randint(1,10, size=(num_of_items)))
# costs of every item (e.g. [2, 4, 8, 8, 4, 2, 5, 2, 4, 9, 7, 2])
W = list(np.random.randint(1,10, size=(num_of_items)))
# total cost upper bound (e.g. 23)
C = np.random.randint(12, 40)
# upper bounds of selling every item (e.g. [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
B = list(np.random.randint(2, 6,size=(num_of_items)))
print('------------- Problem set up -------------')
print('values:',V)
print('costs:',W)
print('budget limit:',C)
print('quantity bounds:',B)

------------- Problem set up -------------
values: [3, 4, 4, 5, 2, 9, 4, 7, 8, 9, 2, 2]
costs: [2, 8, 3, 2, 3, 3, 4, 1, 6, 3, 3, 6]
budget limit: 29
quantity bounds: [4, 3, 3, 5, 4, 2, 2, 5, 3, 2, 2, 2]


## Building CQM
Discrete variables $x_{i}$ are introduced to denote the quantity of the restoring item $U_i$. Notice that there are upper bounds $B_i$ for each item $U_i$, which is an implicit constraint, i.e., 

$$
0\leq x_i\leq B_i,\ \ \ \ i=0,\cdots,N-1.
$$

In [3]:
# ---------- Build QCM ------------
cqm = CQM()

# Create discrete variables
x = [Integer(i, upper_bound=B[i]) for i in range(num_of_items)]

Our target is to maximize the net profit (total profit - total cost), that is, the sum of negative net profit should be minimized:

$$
\min \sum_{i=0}^{N-1} (W_i-V_i)\cdot x_i.
$$

In [4]:
# -------------- Objective Function ------------------
# maximize total net profit
# Add obj to CQM
cqm.set_objective(quicksum((W[i]-V[i])*x[i] for i in range(num_of_items)))

There is only one remaining constraint now: the total cost should not exceed the budget limit:

$$
\sum_{i=0}^{N-1} W_i\cdot x_i \leq C.
$$

In [5]:
# -------------- Constraints -----------------
# no exceeding total weight
cqm.add_constraint( quicksum(W[i]*x[i] for i in range(num_of_items))<=C,
                    label = 'budget limit' )

'budget limit'

## Submit to CQM Solver

In [6]:
# -------------- Submit to CQM sampler ---------------
cqm_sampler = LeapHybridCQMSampler()
sampleset = cqm_sampler.sample_cqm(cqm, label = 'Inventory Demo')

## Results

In [7]:
# print set up again to compare
print('------------- Problem set up -------------')
print('values:',V)
print('costs:',W)
print('budget limit:',C)
print('quantity bounds:',B)

# -------------- Process the results ---------------
print('------------- Solution -------------')
feasible_sols = sampleset.filter(lambda row: row.is_feasible == True)
if not len(feasible_sols):
    print("\nNo feasible solution found.\n")
else:
    sol = [int(feasible_sols.first.sample[i]) for i in range(num_of_items)]
    print('best solution: ', sol)
    print('Total cost: ', quicksum(W[i]*sol[i] for i in range(num_of_items)))
    print('Total value: ', quicksum(V[i]*sol[i] for i in range(num_of_items)))


------------- Problem set up -------------
values: [3, 4, 4, 5, 2, 9, 4, 7, 8, 9, 2, 2]
costs: [2, 8, 3, 2, 3, 3, 4, 1, 6, 3, 3, 6]
budget limit: 29
quantity bounds: [4, 3, 3, 5, 4, 2, 2, 5, 3, 2, 2, 2]
------------- Solution -------------
best solution:  [1, 0, 0, 5, 0, 2, 0, 5, 0, 2, 0, 0]
Total cost:  29
Total value:  99
